In [17]:
import os
import pandas as pd
import glob

from pathlib import Path

In [28]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
print(data_dir)

C:\Users\Silbe\Desktop\Simulations\source_code\data


In [34]:
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")

In [74]:
balance_dfs=[]

for file in balance_files:
    new_df = pd.read_csv(file)
    columns = ['player_name','player_type','beginning_balance'] + ['game_' + str(num) for num in range(new_df.shape[1] - 3)]
    new_df.columns = columns
    
all_balances = pd.concat([new_df])

In [75]:
all_balances

,player_name,player_type,beginning_balance,game_0,game_1,game_2,game_3,game_4,game_5,game_6,...,game_990,game_991,game_992,game_993,game_994,game_995,game_996,game_997,game_998,game_999
0,players - standard - 1,AlwaysCallPlayer,1000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,players - standard - 2,AlwaysCallPlayer,1000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,players - standard - 3,AlwaysCallPlayer,1000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,players - standard - 4,AlwaysCallPlayer,1000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,players - standard - 5,AlwaysCallPlayer,1000,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,players - smart - 6,SmartPlayer,1000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
